# Using Semantic Kernel with MCP Servers

In this ntoebook, we'll connect a simple SK Agent to an MCP Server.

The full example can be found at [microsoft/OpenAIWorkshop](https://github.com/microsoft/OpenAIWorkshop/).


First, let's start our MCP server.

Open a new terminal and run:

```console
pip install fastmcp
cd .\03-semantic-kernel-mcp\
python .\mcp_server.py
```

The server should come up like this:

```
INFO:     Started server process [49488]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
```

Great, ready to go, let's connect SK to it:

In [9]:
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPSsePlugin

# Set up the SSE plugin for the MCP service.
contoso_mcp_plugin = MCPSsePlugin(
    name="ContosoMCP",
    description="Contoso MCP Plugin",
    url="http://localhost:8000/sse", # Replace this if you're not running it locally
    headers={"Content-Type": "application/json"},
    timeout=30,
)

# Open the SSE connection so tools/prompts are loaded
await contoso_mcp_plugin.connect()

# Now create our agent and plug in the MCP plugin
agent = ChatCompletionAgent(
    service=AzureChatCompletion(),
    name="ChatBot",
    instructions="You are a helpful assistant. You can use multiple tools to find information "
    "and answer questions. Review the tools available under the MCPTools plugin "
    "and use them as needed. You can also ask clarifying questions if the user is not clear.",
    plugins=[contoso_mcp_plugin],
)

# Same as prior in our workshop 
thread: ChatHistoryAgentThread = None
user_messages = [
    "I noticed my last invoice was higher than usual—can you help me understand why and what can be done about it?",
    "My customer id is 42",
    ]

for user_message in user_messages:
    print("*** User:", user_message)
    response = await agent.get_response(messages=user_message, thread=thread)
    thread = response.thread
    print("*** Agent:", response.content)



*** User: I noticed my last invoice was higher than usual—can you help me understand why and what can be done about it?
*** Agent: I can help you with that. To get started, I need to know your customer ID or any other information about your account that can help me locate the right details.
*** User: My customer id is 42
*** Agent: I reviewed your subscription details and found information regarding your invoices. Your subscription, "Contoso International Roaming," is active and part of an add-on service for travelers.

Here are some key points regarding your recent invoices:

1. **Invoice Amounts**:
   - **Invoice #460**: $129.71 with an outstanding due to a failed payment via Apple Pay. 
   - **Invoice #462**: $65.60 with outstanding payment due to a failed PayPal transaction.
   - **Invoice #457**: $99.12 is still outstanding due to a failed ACH payment.

These outstanding payments are likely contributing to your higher than usual invoice for the current billing period.

To address 